In [1]:
%matplotlib inline
from IPython.core.display import HTML
from IPython.display import YouTubeVideo
from pandas_datareader import data, wb

import os
import pandas as pd
import numpy as np
import datetime

path1 = os.path.join(os.getcwd(),'style-table.css')
path2 = os.path.join(os.getcwd(),'style-notebook.css')

css = open(path1).read() + open(path2).read()
HTML('<style>{}</style>'.format(css))

### stacking

In [10]:
# make dataframe from numpy array
# by using pd.Index([]) you can name the indexes

df1 = pd.DataFrame( np.arange(8).reshape(2,4),
                    index=pd.Index(['LA','SF'],name='city'),
                    columns=pd.Index(['A','B','C','D'],name='letter')
                    )
df1

letter,A,B,C,D
city,,,,
LA,0,1,2,3
SF,4,5,6,7


In [14]:
# the column index goes downward
# we use stack to pivot the columns into rows
# letter becomes a second index for the rows

df_stacked = df1.stack()
df_stacked

city  letter
LA    A         0
      B         1
      C         2
      D         3
SF    A         4
      B         5
      C         6
      D         7
dtype: int32

### unstacking

In [15]:
# move the second index back upward (aka unstack it)

df_stacked.unstack()

letter,A,B,C,D
city,,,,
LA,0,1,2,3
SF,4,5,6,7


In [16]:
# we can explicitly specify which index to unstack (move up)

df_stacked.unstack('letter')

letter,A,B,C,D
city,,,,
LA,0,1,2,3
SF,4,5,6,7


In [17]:
# we can explicitly specify which index to unstack (move up)

df_stacked.unstack('city')

city,LA,SF
letter,,
A,0,4
B,1,5
C,2,6
D,3,7


### unstacking and stacking in one operation

In [21]:
# make 2 series and concatenate them into a dataframe

ser1 = pd.Series([0,1,2],index=['A','B','C'])
ser2 = pd.Series([3,4,5],index=['D','E','F'])

df = pd.concat([ser1,ser2],keys=['Alpha','Beta'])
df

Alpha  A    0
       B    1
       C    2
Beta   D    3
       E    4
       F    5
dtype: int64

In [22]:
# unstack the second index

df.unstack()

,A,B,C,D,E,F
Alpha,0,1,2,NaN,NaN,NaN
Beta,NaN,NaN,NaN,3,4,5


In [24]:
# unstacking and stacking again, does not keep nulls

df.unstack().stack()

Alpha  A    0
       B    1
       C    2
Beta   D    3
       E    4
       F    5
dtype: float64

In [30]:
# unstacking and stacking again
# the dropna argument can keep the null values

df.unstack().stack(dropna=False)

Alpha  A     0
       B     1
       C     2
       D   NaN
       E   NaN
       F   NaN
Beta   A   NaN
       B   NaN
       C   NaN
       D     3
       E     4
       F     5
dtype: float64

### pivoting

In [32]:
# pivot usage depends on what you need
# ask: what do I want to know?
# many times you can use stack/unstack/groupby instead

In [21]:
# create a 3x12 test dataframe
# concatenate 3 series

ser1 = pd.Series( pd.date_range(start=datetime.datetime(2000,1,1),end=datetime.datetime(2000,1,12)))
ser2 = pd.Series( 3*['A'] + 3*['B'] + 3*['C'] + 3*['D'] )
ser3 = pd.Series( np.random.randn(12) )

df = pd.concat([ser1,ser2,ser3],axis=1,
              keys=['date','variable','value'])
df

,date,variable,value
0,2000-01-01,A,-1.132647
1,2000-01-02,A,-0.751366
2,2000-01-03,A,-1.751545
3,2000-01-04,B,-0.767149
4,2000-01-05,B,0.141392
5,2000-01-06,B,0.021544
6,2000-01-07,C,0.381538
7,2000-01-08,C,1.519145
8,2000-01-09,C,1.181657
9,2000-01-10,D,-0.266951


In [22]:
# make a pivot table with pd.pivot()
# arguments: row index , column index , fill value

df_pivoted = df.pivot('date','variable','value')
df_pivoted

variable,A,B,C,D
date,,,,
2000-01-01,-1.132647,NaN,NaN,NaN
2000-01-02,-0.751366,NaN,NaN,NaN
2000-01-03,-1.751545,NaN,NaN,NaN
2000-01-04,NaN,-0.767149,NaN,NaN
2000-01-05,NaN,0.141392,NaN,NaN
2000-01-06,NaN,0.021544,NaN,NaN
2000-01-07,NaN,NaN,0.381538,NaN
2000-01-08,NaN,NaN,1.519145,NaN
2000-01-09,NaN,NaN,1.181657,NaN


### show duplicates in dataframes

In [23]:
df = pd.DataFrame( {
        'key1' : 2*['A'] + 3*['B'],
        'key2' : [2,2,2,3,3]} )
df

,key1,key2
0,A,2
1,A,2
2,B,2
3,B,3
4,B,3


In [25]:
# show if rows are duplicates of previous rows
# duplicates will show as True

df.duplicated()

0    False
1     True
2    False
3    False
4     True
dtype: bool

### drop duplicates in dataframes

In [27]:
# drop any duplicate rows

df.drop_duplicates()

,key1,key2
0,A,2
2,B,2
3,B,3


In [28]:
# drop duplicates, only occuring in a selected column

df.drop_duplicates('key1')

,key1,key2
0,A,2
2,B,2


In [29]:
# drop duplicates, only occuring in a selected column

df.drop_duplicates('key2')

,key1,key2
0,A,2
3,B,3


In [30]:
# drop duplicates, only occuring in a selected column
# keep the last duplicate encountered, instead of the first

df.drop_duplicates('key1',keep='last')

,key1,key2
1,A,2
4,B,3


### mapping in dataframes

In [ ]:
# map() is good for elementwise transformations
# map() is good for and data cleaning operations

In [31]:
df = pd.DataFrame( { 'city' : ['Alma','Brian Head','Fox Park'],
                'altitude'  : [12000,34677,87646] } )
df

,altitude,city
0,12000,Alma
1,34677,Brian Head
2,87646,Fox Park


In [37]:
# we want to add a state column
# we can make a state dictionary and map() it to df.city

state_map = {    'Alma':'Colorado' , 
          'Brian Head' : 'Utah',
            'Fox Park' : 'Wyoming'   }

df['state'] = df.city.map(state_map)
df

,altitude,city,state
0,12000,Alma,Colorado
1,34677,Brian Head,Utah
2,87646,Fox Park,Wyoming


### replacing values in series

In [41]:
ser1 = pd.Series([1,2,3,4,1,2,3,4])

# replace every 1 by a null value

ser1.replace(1,np.nan)

0   NaN
1     2
2     3
3     4
4   NaN
5     2
6     3
7     4
dtype: float64

In [45]:
# replace every 1 by 100 and 4 by 400 respectively

ser1.replace([1,4],[100,400])

0    100
1      2
2      3
3    400
4    100
5      2
6      3
7    400
dtype: int64

In [47]:
# replace every 4 by a null value and every 2 by a 20
# pass dictionary instead, replaces key with value

ser1.replace({4:np.nan,2:20})

0     1
1    20
2     3
3   NaN
4     1
5    20
6     3
7   NaN
dtype: float64

### renaming the indexes in a dataframe

In [50]:
# make a dataframe from a 3x4 numpy array
# add labels for the row indexes
# add labels for the column indexes

df = pd.DataFrame( np.arange(12).reshape(3,4) ,
                    index=['NY','LA','SF'],
                  columns=['A','B','C','D']
                 )
df

,A,B,C,D
NY,0,1,2,3
LA,4,5,6,7
SF,8,9,10,11


In [78]:
df.index

Index(['ny', 'la', 'sf'], dtype='object')

In [79]:
# map the str.lower function onto df's row indexes
# map the str.lower function onto df's column indexes

df.index = df.index.map( str.lower )
df.columns = df.columns.map( str.lower )

df

,a,b,c,d
ny,0,1,2,3
la,4,5,6,7
sf,8,9,10,11


In [80]:
# rename row indexes to same thing but title case
# rename column indexes to same thing but upper case

df.rename( index=str.title , columns=str.upper )

,A,B,C,D
Ny,0,1,2,3
La,4,5,6,7
Sf,8,9,10,11


In [81]:
# rename 1 row index and 1 column index
# use a 'translation dictionary'

df.rename( index={ 'ny' : 'NEW YORK'},
         columns={ 'a'  : 'ALPHA' })


,ALPHA,b,c,d
NEW YORK,0,1,2,3
la,4,5,6,7
sf,8,9,10,11


In [84]:
# same as above but make effects permanent with inplace=True
# rename 1 row index and 1 column index
# use a 'translation dictionary'

df.rename( index={ 'ny' : 'NEW YORK'},
         columns={ 'a'  : 'ALPHA' },
         inplace=True)
df

,ALPHA,b,c,d
NEW YORK,0,1,2,3
la,4,5,6,7
sf,8,9,10,11


### binning

In [9]:
years = [1990,1991,1992,2008,2012,2015,1987,2008,1999]

decade_bins = np.arange(1960,2030,10)

In [28]:
# pd.cut(list to be sorted into bins,number or list of bins)
# pd.cut sorts everything in years list into decade bins

decade_category = pd.cut(years,decade_bins)
decade_category

[(1980, 1990], (1990, 2000], (1990, 2000], (2000, 2010], (2010, 2020], (2010, 2020], (1980, 1990], (2000, 2010], (1990, 2000]]
Categories (6, object): [(1960, 1970] < (1970, 1980] < (1980, 1990] < (1990, 2000] < (2000, 2010] < (2010, 2020]]

In [29]:
# output notation: (1990, 2000]
# (1990 means this bin does not include the number 1990
# 2000] means this bin includes the number 2000

In [30]:
# show start-end points for all categories (or bins)

decade_category.categories

Index(['(1960, 1970]', '(1970, 1980]', '(1980, 1990]', '(1990, 2000]',
       '(2000, 2010]', '(2010, 2020]'],
      dtype='object')

In [31]:
# show how many items from years each bin contains

pd.value_counts(decade_category)

(1990, 2000]    3
(2010, 2020]    2
(2000, 2010]    2
(1980, 1990]    2
(1970, 1980]    0
(1960, 1970]    0
dtype: int64

In [27]:
# cut years into just 2 bins
# define precision to be within +/- 1 year
# result: 2 bins 1987-2001 and 2001-2015

pd.cut(years,2,precision=1)

[(1987, 2001], (1987, 2001], (1987, 2001], (2001, 2015], (2001, 2015], (2001, 2015], (1987, 2001], (2001, 2015], (1987, 2001]]
Categories (2, object): [(1987, 2001] < (2001, 2015]]